**Installing necessary packages and importing necessary libraries**

In [1]:
!pip install -q simpletransformers sacremoses datasets sacrebleu evaluate torch accelerate tqdm
import numpy as np
import pandas as pd
import datasets, evaluate, torch, warnings
warnings.filterwarnings('ignore')
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from accelerate import Accelerator
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, AdamW, get_scheduler, pipeline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.8

**Reading the dataset**

In [2]:
df = pd.read_csv ('/content/drive/MyDrive/por.txt', sep ='\t', header = None)
df.sample(5)

,0,1,2
84204,I'll take a nap after lunch.,Vou tirar uma soneca depois do almoço.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
102634,I wanted to say that I'm sorry.,Eu queria dizer que sinto muito.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2206,Get changed.,Troque de roupa.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
138262,“Where are you?” “I’m here.”,"Onde está você? ""Estou aqui.""",CC-BY 2.0 (France) Attribution: tatoeba.org #9...
85348,That's what Tom had in mind.,Isso é o que Tom tinha em mente.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


**Removing the unnecessary column**

In [3]:
df = df.drop (2, axis =1)
df.sample(3)

,0,1
15084,Here we go again.,E lá vamos nós outra vez.
149763,I'm the third youngest student in my class.,Eu sou o terceiro aluno mais novo da minha cla...
73291,Tom opened his mouth wide.,Tom abriu bem a boca.


**Checking for missing values**

In [4]:
df.isnull().sum()

0    0
1    0
dtype: int64

**No missing values found**

**Checking for and removing duplicates**

In [5]:
print (df.shape)
df = df.drop_duplicates()
print (df.shape)

(168903, 2)
(168903, 2)


**No duplicates found**

**Renaming columns and making an index column**

In [6]:
df = df.rename(columns={0:'en',1:'pt'})
df['id'] = np.arange (0,168903)
df.sample(3)

,en,pt,id
35721,You need to help me.,Vocês precisam me ajudar.,35721
122407,You didn't need to eat so quickly.,Você não precisava comer tão rápido.,122407
88028,You may leave your bag here.,Vocês podem deixar a bolsa aqui.,88028


**Making train, test and validation splits**

In [7]:
train_old, validation = train_test_split (df, test_size =0.2, random_state =1)
train, test = train_test_split (train_old, test_size =0.2, random_state =1)

**Converting the dataset to arrows format for faster processing and removing the index column just generated**

In [8]:
train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)
validation = Dataset.from_pandas(validation)

train = train.remove_columns (['__index_level_0__'])
test = test.remove_columns (['__index_level_0__'])
validation = validation.remove_columns (['__index_level_0__'])

**Adding the 'translation' key containing 'en' and 'pt' columns to the dataset and then removing the 'en' and 'pt' columns previously present**

In [9]:
train= train.map(lambda example: {'translation': {'en': example['en'], 'pt': example['pt']}, 'id': example['id']})
test= test.map(lambda example: {'translation': {'en': example['en'], 'pt': example['pt']}, 'id': example['id']})
validation= validation.map(lambda example: {'translation': {'en': example['en'], 'pt': example['pt']}, 'id': example['id']}
                           )
train = train.remove_columns(['en', 'pt'])
test = test.remove_columns(['en', 'pt'])
validation = validation.remove_columns(['en', 'pt'])

Map:   0%|          | 0/108097 [00:00<?, ? examples/s]

Map:   0%|          | 0/27025 [00:00<?, ? examples/s]

Map:   0%|          | 0/33781 [00:00<?, ? examples/s]

**Merging the 3 sets and converting them to dictionary format**

In [10]:
ds = DatasetDict()

ds['train'] = train
ds['test'] = test
ds['validation'] = validation

ds

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 108097
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 27025
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 33781
    })
})

**Downloading the suitable pretrained model for fine-tuning**

In [11]:
model_checkpoint = "Helsinki-NLP/opus-mt-tc-big-en-pt"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

tokenizer_config.json:   0%|          | 0.00/337 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/803k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/825k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

**Defining tokenization function and applying it to the dataset**

In [12]:
max_length = 128

def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["pt"] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=max_length, truncation=True)
    return model_inputs

tokenized_datasets = ds.map(preprocess_function, batched=True, remove_columns=ds["train"].column_names)

Map:   0%|          | 0/108097 [00:00<?, ? examples/s]

Map:   0%|          | 0/27025 [00:00<?, ? examples/s]

Map:   0%|          | 0/33781 [00:00<?, ? examples/s]

**Initializing the model, data collator, evaluation metric, optimizer & accelerator**

In [13]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
metric = evaluate.load("sacrebleu")
optimizer = AdamW(model.parameters(), lr=2e-5)
accelerator = Accelerator()

config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/465M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

**Preparing train and evaluation data loaders**

In [14]:
tokenized_datasets.set_format("torch")

train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, collate_fn=data_collator,
    batch_size=16)

eval_dataloader = DataLoader(tokenized_datasets["validation"], collate_fn=data_collator, batch_size=16)

model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader,
                                                                          eval_dataloader)

**Defining training arguments**

In [15]:
num_train_epochs = 2
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

**Defining the name and output directory of trained model**

In [16]:
model_name = "marian-finetuned-kde4-en-to-pt-accelerate"

output_dir = "/content/drive/MyDrive/nlp/translation/translation/"

**Defining the post processing function**

In [17]:
def postprocess(predictions, labels):
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

**Model Training**

In [18]:
progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    model.eval()
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=128,
            )
        labels = batch["labels"]

        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(generated_tokens)
        labels_gathered = accelerator.gather(labels)

        decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=decoded_preds, references=decoded_labels)

    results = metric.compute()
    print(f"epoch {epoch}, BLEU score: {results['score']:.2f}")

    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)

  0%|          | 0/13514 [00:00<?, ?it/s]

  0%|          | 0/2112 [00:00<?, ?it/s]

epoch 0, BLEU score: 61.50


  0%|          | 0/2112 [00:00<?, ?it/s]

epoch 1, BLEU score: 62.03


In [24]:
ds['test']['translation'][9:10]

[{'en': 'Everything would be easier if we could float.',
  'pt': 'Tudo seria mais fácil se pudéssemos flutuar.'}]

**Taking custom input from user and translating into portuguese**

In [25]:
model_checkpoint = "/content/drive/MyDrive/nlp/translation/translation/"
translator = pipeline("translation", model=model_checkpoint)
text = input ("Enter the text: ")
output = translator(text)
translated_text = output[0]['translation_text']
print("This text in portuguese is: ", translated_text)

Enter the text: Everything would be easier if we could float.
This text in portuguese is:  Tudo seria mais fácil se pudéssemos flutuar.
